In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

Using TensorFlow backend.
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
set_session(tf.Session(config=config))

In [3]:
ft = fastText.load_model("/home1/zishan/raghav/wiki.hi.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [5]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [6]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [7]:
def train_dev_sentences(filetrain, filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]
    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)

In [8]:
train_file = '/home1/zishan/raghav/Data/train_total.txt'
dev_file = '/home1/zishan/raghav/Data/test_total.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(train_file, dev_file, 3)

In [9]:
n_words = 0
for sentence in train_sentences:
    n_words+=len(sentence)
print(n_words)

239628


In [10]:
from collections import Counter
print(Counter(train_labels))
print(labels2Idx)

Counter({0: 1475, 2: 368, 1: 212, 3: 176, 7: 147, 4: 143, 6: 62, 5: 52, 8: 32})
{'SADNESS': 0, 'NO-EMOTION': 1, 'SYMPATHY/PENSIVENESS': 2, 'OPTIMISM': 3, 'JOY': 4, 'ANGER': 5, 'DISGUST': 6, 'FEAR/ANXIETY': 7, 'SURPRISE': 8}


In [11]:
print(train_sentences[:2])
print(train_labels[:2])
print(labels2Idx)
print(len(train_labels))

['इसके बाद से यहां पर भारतीय सेना तैनात रहती है।', 'सरकारी मीडिया ने सोमवार को बताया कि बाढ़ के भंवर में फंस कर फुजिआन प्रांत में ४३, हुनान में ७८ और गुआंगडांग में ३३ लोगों की मौत हो गई।']
[0, 0]
{'SADNESS': 0, 'NO-EMOTION': 1, 'SYMPATHY/PENSIVENESS': 2, 'OPTIMISM': 3, 'JOY': 4, 'ANGER': 5, 'DISGUST': 6, 'FEAR/ANXIETY': 7, 'SURPRISE': 8}
2667


In [12]:
n_labels = len(labels2Idx)

In [13]:
def compile_model_bilstm(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)
    model_concatenated = Flatten()(lstm_block)
    model_concatenated = Dense(100)(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [14]:
def compile_model_bilstm_cnn(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

# Pretraining

In [ ]:
model = compile_model_bilstm_cnn(no_labels = n_labels)

In [ ]:
train_file = '/home1/zishan/raghav/Data/train.txt'
weights_file ='/home1/zishan/raghav/weights/bilstm_3cnn_dropout=0.5.h5'
log_file = '/home1/zishan/raghav/logs/bilstm_3cnn_dropout=0.5.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [ ]:
max_f1 = 0
for epoch in range(200):
    print("Epoch {}".format(epoch))
    model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=1,)

    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = model.predict(testset_features)


    predLabels = results.argmax(axis=-1)
    devLabels = dev_labels
    f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
    r = recall_score(devLabels, predLabels, average='macro')
    p = precision_score(devLabels, predLabels, average='macro')
    a = accuracy_score(devLabels, predLabels)
    
    if f1> max_f1:
        model.save_weights(weights_file)
        with open(log_file,'a+') as f:
            text = str(epoch)+', a: '+str(a) +', f1:' +str(f1) +'\n'
            f.write(text)
        max_f1 = f1

    print(a,f1)

# transfer learning

In [15]:
word_vectors_ft = {}
def process_features_crosslingual(textline, nb_sequence_length, nb_embedding_dims, tokenize=True, transmat = None):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            wv = np.matmul(wv, transmat) # applying transformation on the word vector to make the vector in same space
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [16]:
def sequential_generator_crosslingual(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         transmat:'Matrix to make embeddings in same vector space'= None,
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False,  
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features_crosslingual(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize, transmat = transmat)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
#         print(batch_features_ft.shape, batch_labels)
        yield ([batch_features_ft], batch_labels)

In [17]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               transmat: 'matrix if crosslingual training'=None,
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model_bilstm_cnn(4)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(len(labels2Idx), activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save_weights(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            text = "************\nUnfreezing {}\n****************\n".format(final_model.layers[ulayer[0]].name)
            with open(filename_to_log,'a') as f:
                f.write(text)         
            print(text)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx, transmat = transmat, tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features_crosslingual(dev_sentences[i], nb_sequence_length, nb_embedding_dims, transmat= transmat)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save_weights(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                    to_write = "p:{0}\t r:{1}\t f1(macro):{2}\t acc:{3}\n".format(max_p,max_r,max_f1,max_a)
                    with open(filename_to_log,'a') as f:
                        f.write(to_write)
                        
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        
        to_write = "p:{0}\t r:{1}\t f1(macro):{2}\t acc:{3}\n".format(max_p,max_r,max_f1,max_a)
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)

In [18]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (18,16), (18,3), (18,1)]

strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
# strings = ['suf_td','all_unfreeze', 'gradual_unfreeze']
# strings=['gradual_unfreeze']
unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]
# unfreeze_strategy = [gradual_unfreezing]
# unfreeze_strategy = [ single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]


In [ ]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))
    
    generator = sequential_generator_crosslingual
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 1
    transmat = np.loadtxt('/home1/zishan/raghav/fastText_multilingual/alignment_matrices/hi.txt')
    number_of_epochs = 200
    labels2Id = labels2Idx
    log_file = '/home1/zishan/raghav/logs/tl_semeval_' +string+'.txt' 
    print("log file: %s" %(log_file))
    weights_file='/home1/zishan/raghav/weights/tl_semeval_'+string+'.h5'
    print("save weights file: %s" %(weights_file))
    batch_size=16
    train_file='/home1/zishan/raghav/Data/train_total.txt'
    f1_measure='macro'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    load_weights_file = '/home1/zishan/raghav/weights/pretrain_semeval_bilstm_3cnn.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=3
    
    test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=log_file, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=weights_file,
           batch_size=batch_size,
           unfreezing_strategy = strategy,       
           train_file=train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=True,
           model_weights_file = load_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           transmat = transmat,
           check_for_generator= check_for_generator)

approach: suf_bu
log file: /home1/zishan/raghav/logs/tl_semeval_suf_bu.txt
save weights file: /home1/zishan/raghav/weights/tl_semeval_suf_bu.h5
[(18, 18), (17, 16), (15, 3), (2, 1), (18, 1)]
Test 1/1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 75, 200)      320800      input_6[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_26 (LeakyReLU)      (None, 75, 200)      0           bidirectional_6[0][0]            
__________________________________________________________________________________________________
conv1d_1

/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


167/167 [==============================] - 16s 96ms/step - loss: 1.6038 - acc: 0.5558
model saved. F1 is 0.106143
Test-Data: Prec: 0.174, Rec: 0.126, F1: 0.106, Acc: 0.572
Epoch: 3/200
Epoch 1/1
167/167 [==============================] - 16s 96ms/step - loss: 1.5757 - acc: 0.5576
model saved. F1 is 0.111833
Test-Data: Prec: 0.174, Rec: 0.129, F1: 0.112, Acc: 0.575
Epoch: 4/200
Epoch 1/1
167/167 [==============================] - 16s 96ms/step - loss: 1.5581 - acc: 0.5569
Test-Data: Prec: 0.164, Rec: 0.129, F1: 0.111, Acc: 0.573
Epoch: 5/200
Epoch 1/1
167/167 [==============================] - 16s 95ms/step - loss: 1.5504 - acc: 0.5599
Test-Data: Prec: 0.174, Rec: 0.128, F1: 0.109, Acc: 0.573
Epoch: 6/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5341 - acc: 0.5580
model saved. F1 is 0.113960
Test-Data: Prec: 0.165, Rec: 0.131, F1: 0.114, Acc: 0.575
Epoch: 7/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5337 - acc: 0.

167/167 [==============================] - 16s 94ms/step - loss: 1.5117 - acc: 0.5614
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 51/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5018 - acc: 0.5576
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 52/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5052 - acc: 0.5580
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 53/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5042 - acc: 0.5588
Test-Data: Prec: 0.174, Rec: 0.133, F1: 0.117, Acc: 0.578
Epoch: 54/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5042 - acc: 0.5603
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 55/200
Epoch 1/1
167/167 [==============================] - 16s 97ms/step - loss: 1.5071 - acc: 0.5584
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 56/200
Ep

167/167 [==============================] - 15s 90ms/step - loss: 1.5037 - acc: 0.5618
Test-Data: Prec: 0.174, Rec: 0.133, F1: 0.117, Acc: 0.578
Epoch: 100/200
Epoch 1/1
167/167 [==============================] - 15s 89ms/step - loss: 1.5020 - acc: 0.5591
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 101/200
Epoch 1/1
167/167 [==============================] - 15s 90ms/step - loss: 1.5062 - acc: 0.5580
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 102/200
Epoch 1/1
167/167 [==============================] - 15s 90ms/step - loss: 1.5014 - acc: 0.5606
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 103/200
Epoch 1/1
167/167 [==============================] - 15s 89ms/step - loss: 1.5003 - acc: 0.5580
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 104/200
Epoch 1/1
167/167 [==============================] - 15s 90ms/step - loss: 1.4981 - acc: 0.5588
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 105/

...

167/167 [==============================] - 45s 268ms/step - loss: 0.0998 - acc: 0.9775
Test-Data: Prec: 0.584, Rec: 0.444, F1: 0.491, Acc: 0.702
Epoch: 198/200
Epoch 1/1
167/167 [==============================] - 45s 267ms/step - loss: 0.0622 - acc: 0.9843
Test-Data: Prec: 0.567, Rec: 0.434, F1: 0.477, Acc: 0.693
Epoch: 199/200
Epoch 1/1
167/167 [==============================] - 45s 268ms/step - loss: 0.0976 - acc: 0.9749
Test-Data: Prec: 0.534, Rec: 0.405, F1: 0.442, Acc: 0.687
Epoch: 200/200
Epoch 1/1
167/167 [==============================] - 45s 267ms/step - loss: 0.0974 - acc: 0.9783
Test-Data: Prec: 0.563, Rec: 0.438, F1: 0.479, Acc: 0.701
p:0.6301998722852703	 r:0.4688139107847787	 f1(macro):0.520378956656192	 acc:0.7110778443113772

*****************************************************************************
avg_prec: 0.6301998722852703 total_rec: 0.4688139107847787 total_f1: 0.520378956656192 total_acc: 0.7110778443113772 

approach: suf_td
log file: /home1/zishan/raghav/log

167/167 [==============================] - 16s 93ms/step - loss: 1.5307 - acc: 0.5603
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 13/200
Epoch 1/1
167/167 [==============================] - 16s 93ms/step - loss: 1.5182 - acc: 0.5588
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 14/200
Epoch 1/1
167/167 [==============================] - 16s 93ms/step - loss: 1.5243 - acc: 0.5610
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 15/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5270 - acc: 0.5591
model saved. F1 is 0.121192
Test-Data: Prec: 0.161, Rec: 0.136, F1: 0.121, Acc: 0.579
Epoch: 16/200
Epoch 1/1
167/167 [==============================] - 16s 93ms/step - loss: 1.5291 - acc: 0.5606
Test-Data: Prec: 0.165, Rec: 0.131, F1: 0.114, Acc: 0.575
Epoch: 17/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.5110 - acc: 0.5603
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119,

167/167 [==============================] - 17s 100ms/step - loss: 1.5064 - acc: 0.5614
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 61/200
Epoch 1/1
167/167 [==============================] - 17s 100ms/step - loss: 1.5056 - acc: 0.5606
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 62/200
Epoch 1/1
167/167 [==============================] - 17s 100ms/step - loss: 1.5052 - acc: 0.5599
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 63/200
Epoch 1/1
167/167 [==============================] - 17s 99ms/step - loss: 1.5021 - acc: 0.5591
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 64/200
Epoch 1/1
167/167 [==============================] - 17s 100ms/step - loss: 1.5029 - acc: 0.5603
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 65/200
Epoch 1/1
167/167 [==============================] - 17s 101ms/step - loss: 1.5066 - acc: 0.5595
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 66/2

167/167 [==============================] - 15s 91ms/step - loss: 1.4995 - acc: 0.5614
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 110/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.5012 - acc: 0.5584
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 111/200
Epoch 1/1
167/167 [==============================] - 16s 93ms/step - loss: 1.5035 - acc: 0.5603
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 112/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.4983 - acc: 0.5595
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 113/200
Epoch 1/1
167/167 [==============================] - 16s 93ms/step - loss: 1.4982 - acc: 0.5606
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 114/200
Epoch 1/1
167/167 [==============================] - 16s 93ms/step - loss: 1.5006 - acc: 0.5610
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 115/

167/167 [==============================] - 16s 94ms/step - loss: 1.4998 - acc: 0.5576
Test-Data: Prec: 0.166, Rec: 0.133, F1: 0.117, Acc: 0.576
Epoch: 158/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.5069 - acc: 0.5599
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 159/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.5027 - acc: 0.5588
Test-Data: Prec: 0.174, Rec: 0.131, F1: 0.115, Acc: 0.576
Epoch: 160/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.4964 - acc: 0.5584
Test-Data: Prec: 0.174, Rec: 0.133, F1: 0.117, Acc: 0.578
Epoch: 161/200
Epoch 1/1
167/167 [==============================] - 16s 94ms/step - loss: 1.4955 - acc: 0.5588
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 162/200
Epoch 1/1
167/167 [==============================] - 15s 92ms/step - loss: 1.4985 - acc: 0.5606
Test-Data: Prec: 0.167, Rec: 0.134, F1: 0.119, Acc: 0.578
Epoch: 163/